In [1]:
import tensorflow as tf
from matplotlib import pyplot
import numpy as np
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import time
import os
import tempfile

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# Load the Datasets
path = "LUCAS-Copy1.SOIL_corr (1).csv"
df = read_csv(path, header=0)
select_columns = [col for col in df.columns if col.startswith('spc.')]
df_select_columns = df[select_columns]
print(df_select_columns.head())

NameError: name 'read_csv' is not defined

In [12]:
# Split into input/output columns
# Encode Strings to integers
# Split into Training and Test DatasetsX, y = df.values[:, :-1], df.values[:, -1]
X, y = df_select_columns.values[:, :-1], df_select_columns.values[:, -1]
X = X.astype("float32")
y = LabelEncoder().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=17
)
n_features = X_train.shape[1]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13325, 4199) (5711, 4199) (13325,) (5711,)


In [13]:
# Create the model first time

# Define Model
# 767 Bands
N = 1200
model_1 = Sequential()
model_1.add(
    Dense(
        N,
        activation="relu",
        kernel_initializer="he_normal",
        kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
        input_shape=(n_features,),
    )
)

In [14]:
model_1.add(
    Dense(
        N // 2,
        activation="relu",
        kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
        kernel_initializer="he_normal",
    )
)

In [15]:
model_1.add(
    Dense(
        N // 4,
        activation="relu",
        kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
        kernel_initializer="he_normal",
    )
)

In [16]:
model_1.add(
    Dense(
        N // 8,
        activation="relu",
        kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
        kernel_initializer="he_normal",
    )
)
model_1.add(Dense(1, activation="linear"))

In [17]:
# Define a callback to modify the learning rate dynamically
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.1, patience=10, min_lr=0.0001
)
# In[ ]:

msle = MeanSquaredLogarithmicError()

In [18]:
# Compile Model
model_1.compile(
    optimizer="sgd",
    loss=msle,
    metrics=["msle"],
)
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1200)              5040000   
_________________________________________________________________
dense_6 (Dense)              (None, 600)               720600    
_________________________________________________________________
dense_7 (Dense)              (None, 300)               180300    
_________________________________________________________________
dense_8 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 151       
Total params: 5,986,201
Trainable params: 5,986,201
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Save the initial weights
initial_weights = os.path.join(tempfile.mkdtemp(), "initial_weights")
model_1.save_weights(initial_weights)

In [20]:
##DO ALL YOUR TRAINING USING MODEL_1###
start_time = time.time()
history_1 = model_1.fit(
    X_train,
    y_train,
    epochs=300,
    batch_size=50,
    validation_split=0.3,
    callbacks=[lr_callback],
)
end_time = time.time()
training_time = end_time - start_time

Epoch 1/300
187/187 [==============================] - 6s 29ms/step - loss: 10.5639 - msle: 8.8505 - val_loss: 2.4185 - val_msle: 0.7042
Epoch 2/300
187/187 [==============================] - 5s 28ms/step - loss: 2.4555 - msle: 0.7416 - val_loss: 2.3884 - val_msle: 0.6762
Epoch 3/300
187/187 [==============================] - 5s 28ms/step - loss: 2.4439 - msle: 0.7322 - val_loss: 2.3853 - val_msle: 0.6753
Epoch 4/300
187/187 [==============================] - 5s 28ms/step - loss: 2.4628 - msle: 0.7535 - val_loss: 2.3824 - val_msle: 0.6748
Epoch 5/300
187/187 [==============================] - 5s 28ms/step - loss: 2.4297 - msle: 0.7227 - val_loss: 2.3795 - val_msle: 0.6743
Epoch 6/300
187/187 [==============================] - 5s 28ms/step - loss: 2.4310 - msle: 0.7265 - val_loss: 2.3768 - val_msle: 0.6740
Epoch 7/300
187/187 [==============================] - 5s 28ms/step - loss: 2.4530 - msle: 0.7508 - val_loss: 2.3739 - val_msle: 0.6735
Epoch 8/300
187/187 [==========================

Epoch 61/300
187/187 [==============================] - 5s 27ms/step - loss: 2.2778 - msle: 0.7000 - val_loss: 2.2218 - val_msle: 0.6457
Epoch 62/300
187/187 [==============================] - 5s 27ms/step - loss: 2.2857 - msle: 0.7101 - val_loss: 2.2189 - val_msle: 0.6449
Epoch 63/300
187/187 [==============================] - 5s 26ms/step - loss: 2.2488 - msle: 0.6755 - val_loss: 2.2159 - val_msle: 0.6442
Epoch 64/300
187/187 [==============================] - 5s 27ms/step - loss: 2.2383 - msle: 0.6672 - val_loss: 2.2130 - val_msle: 0.6436
Epoch 65/300
187/187 [==============================] - 5s 26ms/step - loss: 2.2491 - msle: 0.6802 - val_loss: 2.2099 - val_msle: 0.6427
Epoch 66/300
187/187 [==============================] - 5s 26ms/step - loss: 2.3143 - msle: 0.7475 - val_loss: 2.2067 - val_msle: 0.6417
Epoch 67/300
187/187 [==============================] - 5s 26ms/step - loss: 2.2645 - msle: 0.6999 - val_loss: 2.2037 - val_msle: 0.6409
Epoch 68/300
187/187 [===================

187/187 [==============================] - 5s 26ms/step - loss: 1.9598 - msle: 0.5034 - val_loss: 1.9411 - val_msle: 0.4862
Epoch 121/300
187/187 [==============================] - 5s 26ms/step - loss: 1.9734 - msle: 0.5190 - val_loss: 1.9375 - val_msle: 0.4845
Epoch 122/300
187/187 [==============================] - 5s 27ms/step - loss: 1.9576 - msle: 0.5051 - val_loss: 1.9314 - val_msle: 0.4804
Epoch 123/300
187/187 [==============================] - 5s 27ms/step - loss: 1.9811 - msle: 0.5305 - val_loss: 1.9252 - val_msle: 0.4761
Epoch 124/300
187/187 [==============================] - 5s 27ms/step - loss: 1.9584 - msle: 0.5098 - val_loss: 1.9205 - val_msle: 0.4733
Epoch 125/300
187/187 [==============================] - 5s 26ms/step - loss: 1.9202 - msle: 0.4736 - val_loss: 1.9139 - val_msle: 0.4688
Epoch 126/300
187/187 [==============================] - 5s 27ms/step - loss: 1.9016 - msle: 0.4569 - val_loss: 1.9082 - val_msle: 0.4649
Epoch 127/300
187/187 [=========================

187/187 [==============================] - 5s 27ms/step - loss: 1.5671 - msle: 0.2218 - val_loss: 1.5247 - val_msle: 0.1809
Epoch 180/300
187/187 [==============================] - 5s 27ms/step - loss: 1.5438 - msle: 0.2005 - val_loss: 1.5318 - val_msle: 0.1898
Epoch 181/300
187/187 [==============================] - 5s 27ms/step - loss: 1.5424 - msle: 0.2009 - val_loss: 1.5204 - val_msle: 0.1804
Epoch 182/300
187/187 [==============================] - 5s 27ms/step - loss: 1.5357 - msle: 0.1960 - val_loss: 1.5246 - val_msle: 0.1863
Epoch 183/300
187/187 [==============================] - 5s 27ms/step - loss: 1.5125 - msle: 0.1747 - val_loss: 1.7624 - val_msle: 0.4260
Epoch 184/300
187/187 [==============================] - 5s 27ms/step - loss: 1.5350 - msle: 0.1991 - val_loss: 1.5017 - val_msle: 0.1673
Epoch 185/300
187/187 [==============================] - 5s 27ms/step - loss: 1.5084 - msle: 0.1744 - val_loss: 1.5034 - val_msle: 0.1708
Epoch 186/300
187/187 [=========================

187/187 [==============================] - 5s 27ms/step - loss: 1.3330 - msle: 0.0982 - val_loss: 1.5207 - val_msle: 0.2873
Epoch 239/300
187/187 [==============================] - 5s 27ms/step - loss: 1.3374 - msle: 0.1045 - val_loss: 1.3026 - val_msle: 0.0711
Epoch 240/300
187/187 [==============================] - 5s 27ms/step - loss: 1.3365 - msle: 0.1055 - val_loss: 1.3091 - val_msle: 0.0794
Epoch 241/300
187/187 [==============================] - 5s 27ms/step - loss: 1.3258 - msle: 0.0967 - val_loss: 1.2883 - val_msle: 0.0605
Epoch 242/300
187/187 [==============================] - 5s 27ms/step - loss: 1.3466 - msle: 0.1192 - val_loss: 1.4421 - val_msle: 0.2162
Epoch 243/300
187/187 [==============================] - 5s 27ms/step - loss: 1.3358 - msle: 0.1103 - val_loss: 1.5204 - val_msle: 0.2963
Epoch 244/300
187/187 [==============================] - 5s 27ms/step - loss: 1.3204 - msle: 0.0968 - val_loss: 1.3092 - val_msle: 0.0870
Epoch 245/300
187/187 [=========================

187/187 [==============================] - 5s 27ms/step - loss: 1.1837 - msle: 0.0554 - val_loss: 1.1899 - val_msle: 0.0628
Epoch 298/300
187/187 [==============================] - 5s 27ms/step - loss: 1.2105 - msle: 0.0839 - val_loss: 1.1693 - val_msle: 0.0440
Epoch 299/300
187/187 [==============================] - 5s 27ms/step - loss: 1.1950 - msle: 0.0701 - val_loss: 1.2132 - val_msle: 0.0897
Epoch 300/300
187/187 [==============================] - 5s 27ms/step - loss: 1.2126 - msle: 0.0895 - val_loss: 1.2295 - val_msle: 0.1077


In [21]:
# Save Model
model_1.save("MyNN.h5")

In [22]:
y_pred = model_1.predict(X_test)
print(y_pred)
print(" ")
print(y_test)

hist_df = pd.DataFrame(history_1.history)

output_csv_file = "MyFile.csv"

with open(output_csv_file, mode="w") as f:
    hist_df.to_csv(f, index=False)
    f.write("\n")

[[10649.437 ]
 [ 3299.3   ]
 [15184.312 ]
 ...
 [ 6859.6514]
 [ 6789.009 ]
 [ 8940.493 ]]
 
[13671  3598 16891 ... 10010  8091 11792]


In [23]:
print(initial_weights)

/local/scratch/job_91442/tmp18pm0i2d/initial_weights
